![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3.2: Generators: Collections

#### Written by Chick Markley, Stevo Bailey, Edward Wang, and Richard Lin
[chick@berkeley.edu](mailto:chick@berkeley.edu),
[stevo@berkeley.edu](mailto:stevo@berkeley.edu),
[edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Table of Contents

1. [Generators and Collections](#generators-and-collections)
1. [when/elsewhen/other](#fir-golden-model)
1. [Wire vs val and var](#wire)
1. [Exercise](#exercise)
1. [Optional Exercise ](#optional-exercise)


## Set Up

In [8]:
// Run this boilerplate for the necessary imports

import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import $ivy.`org.scalanlp::breeze:0.13.2`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}
import scala.collection.mutable
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.11/0.13.2/breeze_2.11-0.13.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.11/0.13.2/
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/net/sf/opencsv/opencsv/2.3/opencsv-2.3.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-math3/3.2/commons-math3-3.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-math3/3.2/commons-math3-3.2.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.pom
Download

Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/core/1.1.2/core-1.1.2.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/core/1.1.2/core-1.1.2.jar
Downloaded https://repo1.maven.org/maven2/net/sourceforge/f2j/arpack_combined_all/0.1/arpack_combined_all-0.1.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze-macros_2.11/0.13.2/breeze-macros_2.11-0.13.2.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire-macros_2.11/0.13.0/spire-macros_2.11-0.13.0.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.11/0.13.0/spire_2.11-0.13.0.jar.sha1
Downloaded https://repo1.maven.org/maven2/com/github/rwl/jtransforms/2.4.0/jtransforms-2.4.0.jar
Downloaded https://repo1.maven.org/ma

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import $ivy.$                            

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import scala.collection.mutable

defined function getVerilog

## Generators and Collections<a name="generators-and-collections"></a> 
In this section we will focus on the concept of *generators* and the use of scala collections as a tool to implement them. Instead of looking at
Chisel code as an *instance* of circuit, i.e. a description of a particulara circuit, 
we will instead consider it to be a generator of a circuit.

We will start by considering the FIR filter from previous exercises.  

In [17]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


defined class My4ElementFir

This circuit is a simple case of a generator because it can generate versions of this 4-tap filter with
different coefficients.  But what if we want the circuit to have more taps.  We are going to do this in several steps.

- Build a software *Golden Model* of a tap configurable FIR.
- Redesign our test to use this model, and confirm that it works.
- Refactor our My4ElementFir to allow an configurable number of taps.
- Test the new circuit using our new test harness



## An FIR Golden Model<a name="fir-golden-model"></a> 
Below is a Scala software implementation of a FIR circuit.  

In [24]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps.
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

defined class ScalaFirFilter

### Seq
Note that **taps** has become a **Seq[Int]** which means that the user of the class can pass a *Seq* or sequence of arbitrary length when constructing the class.
### Registers
With ```  var pseudoRegisters = List.fill(taps.length)(0)``` we create a **List** that will hold the values from previous cycles.  **List** was chosen because it syntax of adding an element to the head and remove the last element is very simple. Just about any member of the scala collections family could be used.  We are also initializing this list to contain all zeros.
### Poke
Our class adds a poke function/method that emulates putting a new input into the filter and cycling the clock.
### Updating the registers
The line ```pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)``` first uses the *take* method of list to keep the all but the last element of the list, then uses the *::* list concatentation operator to add *value* to the head of the reduced version of the list.
### Computing the output
A simple for loop with an accumulator that sums the multiplication of each element of the list times the corresponding tap coefficient. The line with just *accumulator* returns that value as the function result.

## Adapting our previous test for testing our golden model
We will now use our previous work to confirm that our golden model works.  A bit of 
editing magic takes our previous tests harness and morphs it into

In [29]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

out = 1
out = 5
out = 8
out = 10
out = 16
out = 12


filter: ScalaFirFilter = $sess.cmd23Wrapper$Helper$ScalaFirFilter@10dabe17
out: Int = 12

Executing the previous block demonstrates that our software model returns the same results as My4ElementFir did.

## Rebuilding our test to use our golden model.

In [32]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new My4ElementFir(1, 1, 1, 1)) {
    c => new PeekPokeTester(c) {
        for(i <- 0 until 100) {
            val input = scala.util.Random.nextInt(8)
          
            val goldenModelResult = goldenModel.poke(input)

            poke(c.io.in, input)
          
            expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

            step(1)
        }
    }
}

[info] [0.000] Elaborating design...
[info] [0.003] Done elaborating.
Total FIRRTL Compile Time: 10.8 ms
Total FIRRTL Compile Time: 6.5 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512002068221
test cmd16WrapperHelperMy4ElementFir Success: 100 tests passed in 105 cycles taking 0.021784 seconds
[info] [0.021] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = $sess.cmd23Wrapper$Helper$ScalaFirFilter@4bf38b73
res31_1: Boolean = true

In [31]:
class MyManyElementFir(consts: Seq[Int]) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  // Reference solution
  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U(8.W)
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan(consts.length - 1)
}

defined class MyManyElementFir

In [3]:
// Simple 4-point moving average
Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1))) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}
println(chisel3.Driver.emit(() => new MyManyElementFir(Seq(1, 1, 1, 1))))

[info] [0.002] Elaborating design...
[info] [0.061] Done elaborating.
Total FIRRTL Compile Time: 233.1 ms
Total FIRRTL Compile Time: 17.2 ms
End of dependency graph
Circuit state created
[info] [0.002] SEED 1511979425739
test cmd1WrapperHelperMyManyElementFir Success: 6 tests passed in 10 cycles taking 0.027723 seconds
[info] [0.015] RAN 5 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd1WrapperHelperMyManyElementFir : 
  module cmd1WrapperHelperMyManyElementFir : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<8>, out : UInt<8>}
    
    clock is invalid
    reset is invalid
    io is invalid
    reg regs_1 : UInt, clock with : (reset => (reset, UInt<1>("h00"))) @[cmd1.sc 11:33]
    regs_1 <= io.in @[cmd1.sc 11:33]
    reg regs_2 : 

res2_0: Boolean = true

In [4]:
class MyManyDynamicElementFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      muls += regs(i) * io.consts(i)
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementFir

In [5]:
// Simple 4-point moving average
Driver(() => new MyManyDynamicElementFir(4)) {
  c => new PeekPokeTester(c) {
    /* Set constant values */
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    
    /* Set input value and check output value */
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

[info] [0.000] Elaborating design...
[info] [0.011] Done elaborating.
Total FIRRTL Compile Time: 34.2 ms
Total FIRRTL Compile Time: 18.7 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1511979428198
test cmd3WrapperHelperMyManyDynamicElementFir Success: 6 tests passed in 10 cycles taking 0.013470 seconds
[info] [0.008] RAN 5 CYCLES PASSED


res4: Boolean = true

In [6]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = RegInit(Vec(length - 1, UInt(8.W)), Vec.fill(length - 1)(0.U))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

defined class MyManyDynamicElementVecFir

In [7]:
// Simple 4-point moving average
Driver(() => new MyManyDynamicElementVecFir(4)) {
  c => new PeekPokeTester(c) {
    /* Set constant values */
    poke(c.io.consts(0), 1)
    poke(c.io.consts(1), 1)
    poke(c.io.consts(2), 1)
    poke(c.io.consts(3), 1)
    
    /* Set input value and check output value */
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}
println(chisel3.Driver.emit(() => new MyManyDynamicElementVecFir(4)))

[info] [0.000] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 30.0 ms
Total FIRRTL Compile Time: 22.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1511979429218
test cmd5WrapperHelperMyManyDynamicElementVecFir Success: 6 tests passed in 10 cycles taking 0.013107 seconds
[info] [0.009] RAN 5 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
;buildInfoPackage: chisel3, version: 3.0-SNAPSHOT_2017-07-19, scalaVersion: 2.11.11, sbtVersion: 0.13.15, builtAtString: 2017-07-19 18:56:34.453, builtAtMillis: 1500490594453
circuit cmd5WrapperHelperMyManyDynamicElementVecFir : 
  module cmd5WrapperHelperMyManyDynamicElementVecFir : 
    input clock : Clock
    input reset : UInt<1>
    output io : {flip in : UInt<8>, out : UInt<8>, flip consts : UInt<8>[4]}
    
    clock is invalid
    reset is invalid
    io is invalid
    wire _T_21 : UInt<1>[3] @[cmd5.sc 9:70]
    _T_21 is invalid @[cmd5.sc 9:70]
    _T

res6_0: Boolean = true